In [40]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import zipfile
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import pandas as pd

In [41]:
# !curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
# with zipfile.ZipFile('ImageLibrary.zip', 'r') as zip_ref:
#     zip_ref.extractall()
# !pip install pydot
# !pip install pydotplus
# !pip install graphviz
# !pip install python-graphviz

In [42]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [43]:
num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

In [44]:
image_size = (180, 180)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="training",
    seed=1,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="validation",
    seed=1,
    image_size=image_size,
    batch_size=batch_size,
)

Found 23422 files belonging to 2 classes.
Using 18738 files for training.
Found 23422 files belonging to 2 classes.
Using 4684 files for validation.


In [45]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)

In [46]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [47]:
input_shape=image_size + (3,)
num_classes=2

inputs = keras.Input(shape=input_shape)
# Image augmentation block
x = data_augmentation(inputs)

# Entry block
x = layers.Rescaling(1.0 / 255)(x)
x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)

x = layers.Conv2D(64, 3, padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)

previous_block_activation = x  # Set aside residual

for size in [128, 256, 512, 728]:
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    x = layers.SeparableConv2D(size, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    # Project residual
    residual = layers.Conv2D(size, 1, strides=2, padding="same")(
        previous_block_activation
    )
    x = layers.add([x, residual])  # Add back residual
    previous_block_activation = x  # Set aside next residual
    
x = layers.SeparableConv2D(1024, 3, padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)

x = layers.GlobalAveragePooling2D()(x)
if num_classes == 2:
    activation = "sigmoid"
    units = 1
else:
    activation = "softmax"
    units = num_classes

x = layers.Dropout(0.5)(x)
outputs = layers.Dense(units, activation=activation)(x)

model = keras.Model(inputs, outputs)

In [48]:
# model.summary()
# plot_model(model)

In [49]:
epochs = 10

model.compile(optimizer=keras.optimizers.Adam(1e-3), loss="binary_crossentropy", metrics=["accuracy"])

model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/10
586/586 [==============================] - 156s 262ms/step - loss: 0.6427 - accuracy: 0.6532 - val_loss: 0.9097 - val_accuracy: 0.5450
Epoch 2/10
586/586 [==============================] - 156s 266ms/step - loss: 0.4785 - accuracy: 0.7729 - val_loss: 0.6730 - val_accuracy: 0.6684
Epoch 3/10
586/586 [==============================] - 158s 270ms/step - loss: 0.3759 - accuracy: 0.8307 - val_loss: 0.3107 - val_accuracy: 0.8730
Epoch 4/10
586/586 [==============================] - 158s 270ms/step - loss: 0.2972 - accuracy: 0.8747 - val_loss: 0.5293 - val_accuracy: 0.8057
Epoch 5/10
586/586 [==============================] - 159s 271ms/step - loss: 0.2470 - accuracy: 0.8969 - val_loss: 0.2273 - val_accuracy: 0.9118
Epoch 6/10
586/586 [==============================] - 159s 272ms/step - loss: 0.2196 - accuracy: 0.9079 - val_loss: 0.2506 - val_accuracy: 0.9024
Epoch 7/10
586/586 [==============================] - 158s 270ms/step - loss: 0.1978 - accuracy: 0.9187 - val_loss: 0.2368 -

In [50]:
df = pd.DataFrame()
image_list = []
cat_list = []
dog_list = []

for animal in ['Cat','Dog']:
    for i in range(1,6):
        try:
            img = keras.preprocessing.image.load_img(
                "PetImages/"+animal+"/"+str(i)+".jpg", target_size=image_size
            )
            img_array = keras.preprocessing.image.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0)  # Create batch axis

            predictions = model.predict(img_array)
            score = predictions[0]
            cat_list.append(1-score[0])
            dog_list.append(score[0])
            
        except:
            cat_list.append('N/A')
            dog_list.append('N/A')
            
        image_list.append(animal+"/"+str(i))
            
df['image'] = image_list
df['% cat'] = cat_list
df['% dog'] = dog_list

df

,image,% cat,% dog
0,Cat/1,0.999802,0.000198
1,Cat/2,0.999985,0.000015
2,Cat/3,0.828661,0.171339
3,Cat/4,0.998566,0.001434
4,Cat/5,0.877141,0.122859
5,Dog/1,0.436351,0.563649
6,Dog/2,N/A,N/A
7,Dog/3,0.019226,0.980774
8,Dog/4,0.000037,0.999963
9,Dog/5,0.00197,0.99803
